In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import re
import json
import requests
import time
pd.set_option('display.max_columns',50) # DataFrame truncation 없이 보기

In [2]:
# 이후 전처리 작업에서 매번 API 호출할 필요 없다.
# API로 호출한 데이터를 저장한 csv파일을 읽어와 진핼할 것
# 일단 2020년 3월 26일자 기준 데이터로 전처리를 진행하겠다.
df_all = pd.read_csv('./seoul_salon_03-26-20.csv')

In [3]:
# 데이터 확인
df_all.head(2)

,지역,순번,CGG_CODE,SNT_COB_CODE,YY,UPSO_SNO,SNT_COB_NM,UPSO_GSL_YMD,UPSO_NM,TRDP_AREA,UPSO_SITE_TELNO,BMAN_STDT,BUP_NM,SITE_STDT,ADMDNG_NM,DCB_YMD,DCB_WHY,SNT_UPTAE_NM,ED_FIN_YMD,GAEKSIL,HANSHIL,YANGSIL,CHAIR_NUM,YOKSIL,BALHANSIL_YN,WASHMC_NUM,PERM_NT_NO,KOR_FRGNR_GBN,NTN,SITE_ADDR_RD,SITE_ADDR
0,강남,0,3220000,211,1983,1,미용업(일반),19830627,백민재헤어샵,19.80,02 5451290,20110502,NaN,20180515,청담동,NaN,NaN,일반미용업,20110622.0,0.0,0.0,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,NaN,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩
1,강남,1,3220000,211,1983,1,미용업(일반),19830627,백민재헤어샵,65.67,02 5451290,20110502,NaN,20180515,청담동,NaN,NaN,일반미용업,20110622.0,0.0,0.0,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,NaN,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩


In [4]:
# 컬럼으로 들어간 인덱스 두개를 다시 인덱스로 설정하기
df_all = df_all.set_index(['지역', '순번'])
df_all.head(2)

CGG_CODE  SNT_COB_CODE    YY  UPSO_SNO SNT_COB_NM  UPSO_GSL_YMD  \
지역 순번                                                                    
강남 0    3220000           211  1983         1    미용업(일반)      19830627   
   1    3220000           211  1983         1    미용업(일반)      19830627   

      UPSO_NM  TRDP_AREA UPSO_SITE_TELNO  BMAN_STDT BUP_NM  SITE_STDT  \
지역 순번                                                                   
강남 0   백민재헤어샵      19.80      02 5451290   20110502    NaN   20180515   
   1   백민재헤어샵      65.67      02 5451290   20110502    NaN   20180515   

      ADMDNG_NM  DCB_YMD DCB_WHY SNT_UPTAE_NM  ED_FIN_YMD  GAEKSIL  HANSHIL  \
지역 순번                                                                         
강남 0        청담동      NaN     NaN        일반미용업  20110622.0      0.0      0.0   
   1        청담동      NaN     NaN        일반미용업  20110622.0      0.0      0.0   

       YANGSIL  CHAIR_NUM  YOKSIL BALHANSIL_YN  WASHMC_NUM  \
지역 순번                                                        
강남 0       0.0        3.0     0.0            N         0.0   
   1       0.0        3.0     0.0            N         0.0   

                   PERM_NT_NO KOR_FRGNR_GBN  NTN  \
지역 순번                                              
강남 0   3220000-211-1983-00001           내국인  NaN   
   1   3220000-211-1983-00001           내국인  NaN   

                                       SITE_ADDR_RD  \
지역 순번                                                 
강남 0   서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   
   1   서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   

                             SITE_ADDR  
지역 순번                                   
강남 0   서울특별시 강남구 청담동  133번지 3호  화천회관빌딩  
   1   서울특별시 강남구 청담동  133번지 3호  화천회관빌딩

In [5]:
# 컬럼을 보기 쉽게 변환하기
# 컬럼명 정보가 담긴 csv 파일 읽어오기 
col_convert = pd.read_csv('./seoul_salon_column_info.csv', encoding='cp949')
col_convert

,출력명,출력설명
0,CGG_CODE,시군구코드
1,SNT_COB_CODE,업종코드
2,YY,년도
3,UPSO_SNO,업소일련번호
4,SNT_COB_NM,업종명
5,UPSO_GSL_YMD,신고일자
6,UPSO_NM,업소명
7,TRDP_AREA,면적
8,UPSO_SITE_TELNO,소재지전화번호
9,BMAN_STDT,영업자시작일


In [6]:
# 두 컬럼의 값을 dict 객체로 변환
col_dict = dict(zip(col_convert.출력명, col_convert.출력설명))
col_dict

{'CGG_CODE': '시군구코드',
 'SNT_COB_CODE': '업종코드',
 'YY': '년도',
 'UPSO_SNO': '업소일련번호',
 'SNT_COB_NM': '업종명',
 'UPSO_GSL_YMD': '신고일자',
 'UPSO_NM': '업소명',
 'TRDP_AREA': '면적',
 'UPSO_SITE_TELNO': '소재지전화번호',
 'BMAN_STDT': '영업자시작일',
 'BUP_NM': '법인명',
 'SITE_STDT': '소재지시작일',
 'ADMDNG_NM': '행정동명',
 'DCB_YMD': '폐업일자',
 'DCB_WHY': '폐업사유',
 'SNT_UPTAE_NM': '업태명',
 'ED_FIN_YMD': '위생교육수료일자',
 'GAEKSIL': '객실수',
 'HANSHIL': '한실수',
 'YANGSIL': '양실수',
 'CHAIR_NUM': '의자수',
 'YOKSIL': '욕실수',
 'BALHANSIL_YN': '발한실',
 'WASHMC_NUM': '세탁기수',
 'PERM_NT_NO': '허가(신고)번호',
 'KOR_FRGNR_GBN': '내외국인구분',
 'NTN': '국적',
 'SITE_ADDR_RD': '소재지도로명',
 'SITE_ADDR': '소재지지번'}

In [7]:
# 컬럼명 변경하기
df_all.rename(columns=col_dict, inplace=True)
df_all.head(2)

시군구코드  업종코드    년도  업소일련번호      업종명      신고일자     업소명     면적  \
지역 순번                                                                  
강남 0   3220000   211  1983       1  미용업(일반)  19830627  백민재헤어샵  19.80   
   1   3220000   211  1983       1  미용업(일반)  19830627  백민재헤어샵  65.67   

          소재지전화번호    영업자시작일  법인명    소재지시작일 행정동명  폐업일자 폐업사유    업태명    위생교육수료일자  \
지역 순번                                                                           
강남 0   02 5451290  20110502  NaN  20180515  청담동   NaN  NaN  일반미용업  20110622.0   
   1   02 5451290  20110502  NaN  20180515  청담동   NaN  NaN  일반미용업  20110622.0   

       객실수  한실수  양실수  의자수  욕실수 발한실  세탁기수                허가(신고)번호 내외국인구분   국적  \
지역 순번                                                                          
강남 0   0.0  0.0  0.0  3.0  0.0   N   0.0  3220000-211-1983-00001    내국인  NaN   
   1   0.0  0.0  0.0  3.0  0.0   N   0.0  3220000-211-1983-00001    내국인  NaN   

                                             소재지도로명  \
지역 순번                                                 
강남 0   서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   
   1   서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   

                                 소재지지번  
지역 순번                                   
강남 0   서울특별시 강남구 청담동  133번지 3호  화천회관빌딩  
   1   서울특별시 강남구 청담동  133번지 3호  화천회관빌딩

In [8]:
# 총 데이터 개수 확인
len(df_all)

28159

---
- 업소명에 '?'문자가 있는 부분을 조사해보니 '샾'을 인식하지 못해 물음표로 처리된 것으로 판단.
- '?' 문자를 '샵'으로 변환

In [9]:
# 업소명에 '?' 문자를 '샵' 문자로 치환하기
df_all['업소명'] = list(map(lambda x: x.replace('?', '샵'), df_all['업소명'].values))

---
- 중복값 처리

In [10]:
# 모든 컬럼값이 중복인 row 제거
df_all.drop_duplicates(inplace=True)

# 중복 제거 후 데이터 개수 확인
len(df_all)

28110

In [11]:
# 중복 제거 전 row 수 : 28159 rows
# 중복 제거 후 row 수 : 28110 rows
print('모든 데이터가 일치하여 제거된 중복 데이터 :', 28159-28110, '개')

모든 데이터가 일치하여 제거된 중복 데이터 : 49 개


---
- 폐업점 데이터 처리

In [12]:
# '폐업일자'의 값이 NaN이면 영업중
# '폐업일자'의 값이 NaN이 아니면 폐업된 것으로 판단

# '폐업일자'에 값이 존재하는 폐업된 업장이 얼마나 있는지 확인 
df_all[df_all['폐업일자'].notnull()]['폐업일자']

지역  순번 
강남  5      20200113.0
    7      20130829.0
    8      20150821.0
    21     20180222.0
    25     20190415.0
              ...    
중랑  857    20190827.0
    878    20191119.0
    909    20190802.0
    916    20200213.0
    919    20200206.0
Name: 폐업일자, Length: 8567, dtype: float64

In [13]:
# 폐업한 미용실 정보가 8567개나 된다.

In [14]:
# '폐업일자' 컬럼이 NaN인, 영업중인 행만 불리언 색인 -> df_all에 대입
df_all = df_all[df_all['폐업일자'].isnull()]

# 데이터 개수 확인
len(df_all)

19543

In [15]:
# 폐업점 drop 전 : 28110 rows
# 폐업점 drop 후 : 19543 rows
print('drop된 폐업점 수 :', 28110 - 19543)

drop된 폐업점 수 : 8567


---
- 아래부터는 전처리 작업 완료(x)
- 여러가지 데이터 뽑아 보면서 고민중

In [16]:
# 업종명 종류
set(df_all['업종명'].values)

{'미용업(일반)'}

In [17]:
# 업태명 종류
set(df_all['업태명'].values)

{'기타', '네일아트업', '메이크업업', '숙박업 기타', '일반미용업', '일반이용업'}

In [18]:
# '업태명'이 '네일아트업'으로 등록된 데이터 수 
len(df_all[df_all['업태명'] == '네일아트업'])

184

In [19]:
# 전체 데이터에서 '업소명'에 '네일'이 포함된 이름 뽑아보기
nail_list = list(filter(lambda x: '네일' in x, df_all['업소명'].values))
nail_list

['네일하우스',
 '마리골드 네일',
 '솜씨네일',
 '티나조네일',
 '조은 네일',
 '반디네일호텔오크우드점',
 '네코네일',
 '파스텔네일',
 '코코네일',
 '포쉬네일 가로수길점',
 '네일라인 가로수길점',
 '네일 원',
 '네일탐이나',
 '울랄라 네일',
 '뷰티네일',
 '루나네일',
 '네일앤네일',
 '코코네일',
 '엠네일(M nail)',
 '정현네일',
 '네일카페',
 'L-Nail(엘네일)',
 '네일맑음',
 '네일맑음',
 '제이제이헤어네일(JJ HAIR NAIL)',
 '네일스퀘어',
 '네일애',
 '네일아트 인',
 '으뜸네일샵',
 '으뜸네일샵',
 '네일홀릭',
 'W 에비뉴 헤어살롱 네일',
 '트윙클네일',
 '네일 바이 민(nail by min)',
 '네일트리',
 '아이뷰 속눈썹 네일',
 '오네일 앤 속눈썹',
 '윤헤어&네일',
 '여우별 네일아트',
 '네일 데이',
 '프리티 네일',
 '호박네일',
 '네일온',
 '실버네일',
 '블링블링네일샵',
 '부미 네일 속눈썹 헤어',
 '예그리나네일',
 '네일의여왕',
 '네일라',
 '라임헤어&네일',
 '네일여행',
 '코코로네일',
 '코코로네일',
 '프랜들리네일',
 '짱 네일아트',
 '네일 하니',
 '네일 하니',
 '개봉동 네일 속눈썹 발라버려',
 '개봉동 네일 속눈썹 발라버려',
 '윤호네일',
 '윤호네일',
 '윤호네일',
 '윤호네일',
 '오키네일',
 '오즈네일샵',
 '퀸즈속눈썹네일붙임머리',
 '엣지헤어앤네일',
 '시크릿 네일',
 '윙크헤어&네일',
 '핑크네일',
 '핑크네일.속눈썹',
 '포쉬 네일',
 '포쉬 네일',
 '포쉬 네일',
 '포쉬 네일',
 '더키네일&속눈썹',
 '더네일',
 '네일스토리',
 '루비네일',
 '네일플러스',
 '수앤수 네일아트',
 '수앤수 네일아트',
 '수앤수 네일아트',
 '수앤수 네일아트',
 '비앤비네일',
 '맑은 네일',
 '토마토네일',
 '미스네일',
 '네

In [20]:
# 전체 데이터 중에 '업소명'에 '네일'이 들어가는 데이터 수
len(nail_list)

168